# Instalando as bibliotecas

In [ ]:
'''!pip install plotly
!pip install matplotlib
!pip install seaborn'''

# Importando as bibliotecas

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
import seaborn as sns

# Para ver todas as colunas e apenas o número de linhas configurado

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Lendo o CSV ( caminho do arquivo )

In [ ]:
# Especifique o caminho para o seu arquivo CSV
caminho_arquivo = '../../dados/dados_siape_consolidado/org_exercicio/cadastro_siape_consolidado_org_exercicio_mes_ano_completo-2021-2023.csv'

df_orgaos = pd.read_csv(caminho_arquivo)

# Queremos apenas os anos que têm estagiários registrados: A partir de 2021

In [ ]:
df_orgaos['ano'].unique()

# Vamos unir "ano" e "mês" em apenas 1 coluna

In [ ]:
df_orgaos['ano_mes'] = df_orgaos['ano'].astype(str) + '-' + df_orgaos['mes'].astype(str).str.zfill(2)

# Ordene o DataFrame com base na coluna 'ano_mes'
df_orgaos = df_orgaos.sort_values(by='ano_mes')

# Como vai ficar

In [ ]:
df_orgaos['ano_mes']

# Vamos criar um dicionário para saber exatamente quantos registros filtrados tem em cada período

In [ ]:
def dicionario_servidores_e_estagio_orgao_ano_mes (df_orgaos, orgaos_escolhidos):

    dicionario_orgaos = {}
    
    meses = df_orgaos['ano_mes'].unique()

    for orgao in orgaos_escolhidos:

        display (orgao)
        
        for mes in meses:                
            
            qtd_regime_unico = df_orgaos[
                        df_orgaos["REGIME_JURIDICO"].str.contains("regime juridico unico", na=False, case=False) &
                        df_orgaos["ORG_EXERCICIO"].str.contains(orgao, na=False, case=False) &
                        (df_orgaos['ano_mes'] == mes)]["Contagem"].sum()
            
            qtd_estagiario =  df_orgaos[
                        df_orgaos["REGIME_JURIDICO"].str.contains("estag", na=False, case=False) &
                        df_orgaos["ORG_EXERCICIO"].str.contains(orgao, na=False, case=False) &
                        (df_orgaos['ano_mes'] == mes)]["Contagem"].sum()

            if not orgao in dicionario_orgaos:
                dicionario_orgaos[orgao] = {}

                        
            dicionario_orgaos[orgao][mes] = {'regime_unico': qtd_regime_unico,'estagiarios': qtd_estagiario,'indice_estagio': qtd_estagiario/qtd_regime_unico}

    # Convertendo o dicionário em DataFrame
    df_resultado = pd.DataFrame.from_dict({(i, j): dicionario_orgaos[i][j] 
                                           for i in dicionario_orgaos.keys() 
                                           for j in dicionario_orgaos[i].keys()},
                                          orient='index')

    df_resultado = df_resultado.reset_index().rename(columns={'level_0': 'org_exercicio', 'level_1': 'ano_mes'})
    
    return df_resultado
    





# Aqui vamos gerar um gráfico do dicionário

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

def gerar_grafico_indice_estagio(df, titulo):

    display(Markdown(f'# {titulo}'))

    # Ajustar tamanho da figura para ser mais comprida na horizontal
    plt.figure(figsize=(20, 10))

    for org in df['org_exercicio'].unique():
        df_org = df[df['org_exercicio'] == org]
        x = df_org['ano_mes'].to_numpy()
        y = df_org['indice_estagio'].to_numpy()
        labels = df_org['indice_estagio'].to_numpy()  # Substitua 'quantidade' pelo nome da coluna com a quantidade

        # Linhas conectando os pontos com marcadores
        plt.plot(x, y, label=org, marker='o', linestyle='-')

        # Adicionar rótulos com a quantidade em cima de cada ponto
        for i, txt in enumerate(labels):
            plt.annotate(f'{y[i]:.2f}', (x[i], y[i]), textcoords="offset points", xytext=(0, 5), ha='center')

    plt.title('Índice de estágio ao longo do tempo')
    plt.xlabel('Ano e mês')
    plt.ylabel('Índice de estágio')
    plt.xticks(rotation=35)  # Rotacionar rótulos do eixo X
    plt.legend()

    # Ajustar espaçamento entre subplots
    plt.tight_layout(pad=2.0)

    plt.show()

# Por fim, definir quais são os "órgãos escolhidos" e visualizar o gráfico

In [ ]:
orgaos_escolhidos = [
    'Instituto Nacional do Seguro Social',
    'ministério da economia', 
    'ministerio da fazenda',
    'Advocacia-Geral da União' ,     
    'Ministério da Ciência, Tecnologia, Inovações e Comunicações', 
    'Ministério da Saúde'
]

# Lista dos órgãos sendo carregados
df_todos_orgaos_escolhidos = dicionario_servidores_e_estagio_orgao_ano_mes(df_orgaos, orgaos_escolhidos)

gerar_grafico_indice_estagio (df_todos_orgaos_escolhidos, "Índice estagiários/servidores nos órgãos escolhidos")

In [ ]:
# Salve o DataFrame como um arquivo CSV
df_orgaos.to_csv('/home/carolcardoso/git/ProgramaEstagio/apresentação/Índice - Servidores X Estagiários - Órgãos escolhidos/Índice - Servidores X Estagiários - Órgãos escolhidos.csv', index=False)